## Causal Inference and Machine Learning

You will revisit the exercise in Week 6 about estimating the effect of the mindset intervention (Athey and Wager, 2019). Download the synthetic data `synthetic_mindset_data.csv` from our Week 6 module. More data descriptions are available in our computer lab exercise questions.

In this assignment, you are asked to implement the machine learning estimation of the average treatment effect over all schools using various machine learning estimators.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostClassifier
from tqdm import tqdm

import tensorflow
import random as python_random
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.regularizers import l2

from tensorflow.keras import layers, optimizers, regularizers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

In [ ]:
# Import and clean the data as in the computer lab

data_original = pd.read_csv('synthetic_mindset_data.csv').rename(columns={'Z':'D'})
n = data_original.shape[0]
data = pd.get_dummies(data_original,columns=['C1','XC'])
X = data.iloc[:,3:]
Y = data.Y
D = data.D
schoolid = data.schoolid-1 # then the schoolid starts from 0
K = 76 # Number of schools 
delta_school_dml=np.zeros((K,)) # Initialize the school-specific estimates

# Initialize the dict objects to store estimates.

delta_debias={}
delta_raw={}

### Exercise 1

Estimate the average treatment effects by using the random forest (of 200 trees) algorithm including a variable selection as proposed by Athey and Wager (2019). Note that you can extract the feature importance directly from sklearn. You should always use the debiased ML estimator across clusters, as in the Week 6 computer lab.

In [ ]:
# Initialize models

n_estimators = 500 # Number of trees to estimate 

rf_mu = RandomForestRegressor(
    n_estimators=n_estimators,     
    random_state=42,
)

rf_p = RandomForestClassifier(
    n_estimators=n_estimators,     
    random_state=42,
)

# Estimate the treatment effect for each school

for k in tqdm(range(K)):
    
    # Fit pilot random forests
    
    selector = (schoolid == k)
    rf_mu.fit(X[~selector], Y[~selector])
    rf_p.fit(X[~selector], D[~selector])
    
    # Select the subset of variables with importance higher than their average
    # Repeat this procedure for each school
    
    importances_mu = rf_mu.feature_importances_
    importances_p = rf_p.feature_importances_
    selected_features_mu = X.columns[importances_mu > np.mean(importances_mu)]
    selected_features_p = X.columns[importances_p > np.mean(importances_p)]
    
    # Fit random forests again, using only the selected variables
    
    rf_mu.fit(X.loc[~selector, selected_features_mu], Y[~selector])
    rf_p.fit(X.loc[~selector, selected_features_p], D[~selector])
    Y_res = Y[selector].values-rf_mu.predict(X.loc[selector, selected_features_mu])
    v = D[selector].values-rf_p.predict_proba(X.loc[selector, selected_features_p])[:,1]
    
    # Store the estimated treatment effect for each school
    
    delta_school_dml[k] = np.mean(Y_res*v)/np.mean(v**2) # Debiased estimator


In [ ]:
# Save your estimate here

delta_debias['RF'] = np.mean(delta_school_dml)
print(delta_debias['RF'])

### Exercise 2

Estimate the average treatment effects by using:

- Least-squares boosting for estimation of $\mu^{(0)}(x) = E[Y^{(0)}_i|X_i = x]$, using a learning rate of no more than 0.1.
- AdaBoost for estiamtion of $p(x) = P(D_i = 1|X_i = x)$

You don't need to consider variable selection. Keep the number of bases to no more than 500 for each model.

You should always use the debiased ML estimator across clusters, as in the Week 6 computer lab.

In [ ]:
# Initialize the models

n_estimators = 500 

gb_mu = GradientBoostingRegressor(
    n_estimators=n_estimators, 
    learning_rate=0.1, 
    random_state=42
)

ada_p = AdaBoostClassifier(
    n_estimators=n_estimators, 
    random_state=42
)

# Estimate the treatment effect for each school

for k in tqdm(range(K)):
    
    # Selector for current school
    
    selector = (schoolid == k)

    # Fit models
    
    gb_mu.fit(X[~selector], Y[~selector])
    ada_p.fit(X[~selector], D[~selector])

    # Predict outcomes and treatment probabilities for each school
    
    Y_res = Y[selector].values-gb_mu.predict(X[selector])
    v = D[selector].values-ada_p.predict_proba(X[selector])[:,1]
    
    # Store the estimated treatment effect for each school
    
    delta_school_dml[k] = np.mean(Y_res*v)/np.mean(D[selector].values*v) # Debiased estimator

In [ ]:
# Estimates

delta_debias['Boost']=np.mean(delta_school_dml)
print(delta_debias['Boost'])

### Exercise 3

Estimate the average treatment effects by estimating $\mu^{(0)}(x) = E[Y^{(0)}_i|X_i = x]$ and $p(x) = P(D_i = 1|X_i = x)$ with two separate neural networks.

Use no more than 6 neurons in each hidden layer, no more than 3 hidden layers, and a weight decay penalty parameter no larger than 0.1. Use ReLU for deep neural nets but sigmoid for shallow nets.

You should always use the debiased ML estimator across clusters, as in the Week 6 computer lab.

We perform a Grid Search to find the optimal number of hidden layers and units for each layer.

In [ ]:
# Split the data into training and validation sets

X_train, X_val, Y_train, Y_val, D_train, D_val = train_test_split(X, Y, D, test_size=0.2, random_state=42)

def build_model(hp):
    
    model = Sequential()

    # Choose the number of hidden layers (1, 2, or 3)
    
    num_hidden_layers = hp.Int('num_hidden_layers', 1, 3)

    for i in range(num_hidden_layers):
        
        # Choose the number of neurons (1 to 6)
        
        units = hp.Int(f'units_layer_{i}', min_value=1, max_value=6)

        # Add hidden layers with ReLU activation and L2 regularization
        
        model.add(layers.Dense(units=units, activation='relu',
                               kernel_regularizer=regularizers.l2(0.1)))

    # Output layer with linear activation
    
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )

    return model

# Set up the RandomSearch tuner

tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=10,  
    executions_per_trial=3,  
    directory='my_dir',
    project_name='hidden_layer_tuning'
)

# Perform the search

tuner.search(
    X_train, Y_train,
    epochs=20,  
    batch_size=32,  
    validation_data=(X_val, Y_val),
    verbose=0
)

# Get the best hyperparameters

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters")
print(f"Number of hidden layers: {best_hps.get('num_hidden_layers')}")
for i in range(best_hps.get('num_hidden_layers')):
    print(f"Units in layer {i}: {best_hps.get(f'units_layer_{i}')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")

# Build the model with the best hyperparameters

best_model = tuner.hypermodel.build(best_hps)

# Train the best model

history = best_model.fit(
    X_train, Y_train,
    epochs=50,  
    batch_size=32,
    validation_data=(X_val, Y_val),
    verbose=0
)

# Evaluate the best model on validation data

val_loss, val_mae = best_model.evaluate(X_val, Y_val, verbose=0)
print(f"Validation Loss: {val_loss}, Validation MAE: {val_mae}")

In [ ]:
# Initialize the models

def create_model(input_dim, output_activation, loss, weight_decay=0.1, units_layer_0=4):
    model = Sequential()
    model.add(Dense(units=units_layer_0, activation='relu', input_dim=input_dim, kernel_regularizer=l2(weight_decay)))
    model.add(Dense(units=1, activation=output_activation, kernel_regularizer=l2(weight_decay)))
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    return model

# Models for mu0 and p

model_mu = create_model(input_dim=X.shape[1], output_activation='linear', loss='mean_squared_error', weight_decay=0.1, units_layer_0=4)
model_p = create_model(input_dim=X.shape[1], output_activation='sigmoid', loss='binary_crossentropy', weight_decay=0.1, units_layer_0=4)

delta_school_dml = np.zeros(K)

# Estimate the treatment effect for each school

for k in tqdm(range(K)):
    
    # Selector for the current school
    
    selector = (schoolid == k)

    # Fit models using training data excluding the current school
    
    model_mu.fit(X[~selector], Y[~selector], epochs=50, batch_size=32, verbose=0)
    model_p.fit(X[~selector], D[~selector], epochs=50, batch_size=32, verbose=0)

    # Predict outcomes and treatment probabilities
    
    Y_res = Y[selector].values - model_mu.predict(X[selector]).flatten()
    v = D[selector].values - model_p.predict(X[selector]).flatten()

    # Store the estimated treatment effect for each school
    
    delta_school_dml[k] = np.mean(Y_res * v) / np.mean(D[selector].values * v)

In [ ]:
# Save your estimate here

delta_debias['NN'] = np.mean(delta_school_dml)
print(delta_debias['NN'])

### Exercise 4

Repeat Exercise 1, but now _without_ using debiased methods:

- Estimate the regression functions $\mu^{(0)}(x) = E[Y^{(0)}_i|X_i = x]$ and $p(x) = P(D_i = 1|X_i = x)$ by pooling ALL observations.
- Estimate the treatment effects for each school.
- Average the estimated treatment effects over schools.

In [ ]:
n_estimators = 200

rf_mu = RandomForestRegressor(
    n_estimators=n_estimators,     
    random_state=42,
)

rf_p = RandomForestClassifier(
    n_estimators=n_estimators,     
    random_state=42,
)

delta_school_dml = np.zeros(K)

# Fit the models using observations over all schools

rf_mu.fit(X, Y)
rf_p.fit(X, D)

# Select the subset of variables with importance higher than their average

importances_mu = rf_mu.feature_importances_
importances_p = rf_p.feature_importances_

selected_features_mu = X.columns[importances_mu > np.mean(importances_mu)]
selected_features_p = X.columns[importances_p > np.mean(importances_p)]

# Fit random forests again, using only the selected variables

rf_mu.fit(X.loc[:,selected_features_mu], Y)
rf_p.fit(X.loc[:,selected_features_p], D)


# Estimate the treatment effect for each school

for k in tqdm(range(K)):
    
    selector = (schoolid == k)

    # Calculate the treatment effect
    
    Y_res = Y[selector].values-rf_mu.predict(X.loc[selector, selected_features_mu])
    D_k = D[selector].values
    
    
    # Store the estimated treatment effect for each school
    
    delta_school_dml[k] = np.mean(Y_res * D_k) / np.mean(D_k**2) # Conventional LS estimator

In [ ]:
# Save your estimate here

delta_raw['RF'] = np.mean(delta_school_dml)
print(delta_raw['RF'])